<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
tanh_output_name = "TanhOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name,onnx.TensorProto.FLOAT,[None, None])

# Create output tensor (final result after tanh operation)
tanh_output = onnx.helper.make_tensor_value_info(tanh_output_name,onnx.TensorProto.FLOAT,[None, None])

# Define Tanh node: Y = Tanh(X)
tanh_node = onnx.helper.make_node("Tanh",[input_name],[tanh_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[tanh_node],
    name="Tanh",
    inputs=[input_tensor],
    outputs=[tanh_output]
)

# Create the ONNX model (Tanh available since early opsets, 13 is safe)
model = onnx.helper.make_model(graph,opset_imports=[onnx.helper.make_opsetid("", 13)])
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "tanh.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("tanh.onnx")

def do_tanh(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    # Display results
    print(f"X={x_f}")
    print(f"Result: tanh(X)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)

do_tanh(x)

#--------------------------------------------------------------------
# Non nominal cases (nan, -inf, 0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -1.0, float("inf"), float("-inf"), float("nan")]

for x_val in v:
    x_np = np.array([[x_val],
                     [x_val]], dtype=np.float32)
    do_tanh(x_np)



#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# X = [0, 1, -1]
x_example_1 = np.array([[0.0, 1.0, -1.0]], dtype=np.float32)
do_tanh(x_example_1)

print("\n## Example 2")

# Example 2
# X =
# [[  -2   0 ]
#  [  1    2  ]
#  [ -4    4  ]]
x_example_2 = np.array([
    [-2.0,   0.0],
    [1.0,    2.0],
    [-4.0,   4.0]
], dtype=np.float32)

do_tanh(x_example_2)

print("\n## Example 3")

# Example 3
# X =
# [[  inf   nan   -inf]]
x_example_3 = np.array([
    [float("inf"), float("nan"), float("-inf")]
], dtype=np.float32)

do_tanh(x_example_3)


X=[[1.00000000,2.00000000], [3.00000000,4.00000000]]
Result: tanh(X)=[[0.76159418,0.96402758], [0.99505472,0.99932921]]
X=[[1.00000000], [1.00000000]]
Result: tanh(X)=[[0.76159418], [0.76159418]]
X=[[0.00000000], [0.00000000]]
Result: tanh(X)=[[0.00000000], [0.00000000]]
X=[[-1.00000000], [-1.00000000]]
Result: tanh(X)=[[-0.76159418], [-0.76159418]]
X=[[inf], [inf]]
Result: tanh(X)=[[1.00000000], [1.00000000]]
X=[[-inf], [-inf]]
Result: tanh(X)=[[-1.00000000], [-1.00000000]]
X=[[nan], [nan]]
Result: tanh(X)=[[nan], [nan]]

## Example 1
X=[[ 0.00000000, 1.00000000,-1.00000000]]
Result: tanh(X)=[[ 0.00000000, 0.76159418,-0.76159418]]

## Example 2
X=[[-2.00000000, 0.00000000], [ 1.00000000, 2.00000000], [-4.00000000, 4.00000000]]
Result: tanh(X)=[[-0.96402758, 0.00000000], [ 0.76159418, 0.96402758], [-0.99932921, 0.99932921]]

## Example 3
X=[[ inf, nan,-inf]]
Result: tanh(X)=[[ 1.00000000,        nan,-1.00000000]]


In [3]:
#--------------------------------------------------------------------
# Precision comparison: float32 implementations vs float64 reference
#--------------------------------------------------------------------

def tanh_np_technique_1(x):
    x = x.astype(np.float32)
    y = np.empty_like(x, dtype=np.float32)

    mask = x < 0
    y[mask] = (np.exp(2.0 * x[mask], dtype=np.float32) - 1.0) / \
              (np.exp(2.0 * x[mask], dtype=np.float32) + 1.0)

    y[~mask] = (1.0 - np.exp(-2.0 * x[~mask], dtype=np.float32)) / \
               (1.0 + np.exp(-2.0 * x[~mask], dtype=np.float32))

    return y.astype(np.float32)


def tanh_np_technique_2(x):
    x = x.astype(np.float32)
    return ((np.exp(2.0 * x, dtype=np.float32) - 1.0) /
            (np.exp(2.0 * x, dtype=np.float32) + 1.0)).astype(np.float32)


def tanh_np_technique_3(x):
    x = x.astype(np.float32)
    return ((1.0 - np.exp(-2.0 * x, dtype=np.float32)) /
            (1.0 + np.exp(-2.0 * x, dtype=np.float32))).astype(np.float32)


def tanh_np_technique_4(x):
    x = x.astype(np.float32)
    return ((np.exp(x, dtype=np.float32) - np.exp(-x, dtype=np.float32)) /
            (np.exp(x, dtype=np.float32) + np.exp(-x, dtype=np.float32))).astype(np.float32)

def tanh_np_technique_5(x):
    x = x.astype(np.float32)
    y = np.empty_like(x, dtype=np.float32)
    ax = np.abs(x)

    # Region 1: very small x
    m1 = ax < 2**-12
    y[m1] = x[m1]

    # Region 2: |x| < 0.625  (minimax polynomial)
    m2 = (ax >= 2**-12) & (ax < 0.625)
    x2 = x[m2]
    z = x2 * x2

    # Coefficients inspired by libm (float32)
    y[m2] = x2 + x2 * z * (
        np.float32(-0.333333313) +
        z * (np.float32(0.133333340) +
        z * np.float32(-0.053968253))
    )

    # Region 3: 0.625 ≤ |x| < 9
    m3 = (ax >= 0.625) & (ax < 9.0)
    t = np.exp(np.float32(2.0) * ax[m3], dtype=np.float32)
    y[m3] = np.sign(x[m3]) * (np.float32(1.0) - np.float32(2.0) / (t + np.float32(1.0)))

    # Region 4: saturation
    m4 = ax >= 9.0
    y[m4] = np.sign(x[m4])

    return y.astype(np.float32)

def tanh_np_technique_6(x):
    x = np.asarray(x, dtype=np.float32)

    # Clamp thresholds used by Eigen/MLAS (non-FMA path)
    clamp = np.float32(7.90531110763549805)

    # tiny threshold where tanh(x) ~ x
    tiny = np.float32(0.0004)

    # clamp
    x_c = np.minimum(np.maximum(x, -clamp), clamp)

    # polynomial coefficients (Eigen/MLAS)
    a1  = np.float32(4.89352455891786e-03)
    a3  = np.float32(6.37261928875436e-04)
    a5  = np.float32(1.48572235717979e-05)
    a7  = np.float32(5.12229709037114e-08)
    a9  = np.float32(-8.60467152213735e-11)
    a11 = np.float32(2.00018790482477e-13)
    a13 = np.float32(-2.76076847742355e-16)

    b0 = np.float32(4.89352518554385e-03)
    b2 = np.float32(2.26843463243900e-03)
    b4 = np.float32(1.18534705686654e-04)
    b6 = np.float32(1.19825839466702e-06)

    x2 = x_c * x_c

    # numerator polynomial p(x)
    p = a13
    p = p * x2 + a11
    p = p * x2 + a9
    p = p * x2 + a7
    p = p * x2 + a5
    p = p * x2 + a3
    p = p * x2 + a1
    p = p * x_c

    # denominator polynomial q(x)
    q = b6
    q = q * x2 + b4
    q = q * x2 + b2
    q = q * x2 + b0

    y = p / q

    # tiny x => tanh(x) ~ x
    y = np.where(np.abs(x) < tiny, x, y)

    return y.astype(np.float32)





def compare_precision_vs_fp64(x_fp64):
    print("\n===================================================")
    print(f"Input X (float64) = {x_fp64.flatten()}")
    print("===================================================")

    # Reference: float64 numpy tanh
    y_ref = np.tanh(x_fp64)  # float64 reference

    # Cast input to float32 for tested implementations
    x_f32 = x_fp64.astype(np.float32)

    results = {
        "ONNX Tanh (ORT float32)": session.run(None, {input_name: x_f32})[0].astype(np.float32),
        "Technique 1 (branching: exp(2x) if x<0 else exp(-2x))": tanh_np_technique_1(x_f32),
        "Technique 2 (single formula exp(2x) / (exp(2x)+1))": tanh_np_technique_2(x_f32),
        "Technique 3 (single formula exp(-2x) / (1+exp(-2x)))": tanh_np_technique_3(x_f32),
        "Technique 4 (classic: (exp(x)-exp(-x))/(exp(x)+exp(-x)))": tanh_np_technique_4(x_f32),
        "Technique 5 (libm-like piecewise polynomial + exp clamp)": tanh_np_technique_5(x_f32),
        "Technique 6 (ONNX-like:Eigen/MLAS polynomial rational approximation)": tanh_np_technique_6(x_f32),
    }

    for name, y_f32 in results.items():
        y_f64 = y_f32.astype(np.float64)

        abs_err = np.abs(y_f64 - y_ref)
        rel_err = abs_err / (np.abs(y_ref) + np.finfo(np.float64).eps)

        print(f"\n{name}")
        print(f"Y(float32) = {y_f32.flatten()}")
        print(f"abs error vs fp64 = {abs_err.flatten()}")
        print(f"rel error vs fp64 = {rel_err.flatten()}")


#--------------------------------------------------------------------
# Stress test values (chosen to expose numerical limits)
#--------------------------------------------------------------------

test_values_fp64 = np.array([
    0.0,
    1e-8,
    -1e-8,
    1.0,
    -1.0,
    5.0,
    -5.0,
    10.0,
    -10.0,
    20.0,
    -20.0,
    50.0,
    -50.0,
    80.0,
    -80.0,
    88.0,      # near float32 exp overflow
    -88.0,
    100.0,
    -100.0,
    1e3,
    -1e3,
    np.inf,
    -np.inf,
    np.nan
], dtype=np.float64)

# 2-rank tensor as required by the ONNX model
x_test_fp64 = test_values_fp64.reshape(-1, 1)

compare_precision_vs_fp64(x_test_fp64)



Input X (float64) = [ 0.00000000e+00  1.00000000e-08 -1.00000000e-08  1.00000000e+00
 -1.00000000e+00  5.00000000e+00 -5.00000000e+00  1.00000000e+01
 -1.00000000e+01  2.00000000e+01 -2.00000000e+01  5.00000000e+01
 -5.00000000e+01  8.00000000e+01 -8.00000000e+01  8.80000000e+01
 -8.80000000e+01  1.00000000e+02 -1.00000000e+02  1.00000000e+03
 -1.00000000e+03             inf            -inf             nan]

ONNX Tanh (ORT float32)
Y(float32) = [ 0.00000000e+00  9.99999905e-09 -9.99999905e-09  7.61594176e-01
 -7.61594176e-01  9.99909163e-01 -9.99909163e-01  1.00000000e+00
 -1.00000000e+00  1.00000000e+00 -1.00000000e+00  1.00000000e+00
 -1.00000000e+00  1.00000000e+00 -1.00000000e+00  1.00000000e+00
 -1.00000000e+00  1.00000000e+00 -1.00000000e+00  1.00000000e+00
 -1.00000000e+00  1.00000000e+00 -1.00000000e+00             nan]
abs error vs fp64 = [0.00000000e+00 9.48953130e-16 9.48953130e-16 2.03366546e-08
 2.03366546e-08 4.17412328e-08 4.17412328e-08 4.12230727e-09
 4.12230727e-09 0

/tmp/ipykernel_5140/348396083.py:21: RuntimeWarning: overflow encountered in exp
  return ((np.exp(2.0 * x, dtype=np.float32) - 1.0) /
/tmp/ipykernel_5140/348396083.py:22: RuntimeWarning: overflow encountered in exp
  (np.exp(2.0 * x, dtype=np.float32) + 1.0)).astype(np.float32)
/tmp/ipykernel_5140/348396083.py:21: RuntimeWarning: invalid value encountered in divide
  return ((np.exp(2.0 * x, dtype=np.float32) - 1.0) /
/tmp/ipykernel_5140/348396083.py:27: RuntimeWarning: overflow encountered in exp
  return ((1.0 - np.exp(-2.0 * x, dtype=np.float32)) /
/tmp/ipykernel_5140/348396083.py:28: RuntimeWarning: overflow encountered in exp
  (1.0 + np.exp(-2.0 * x, dtype=np.float32))).astype(np.float32)
/tmp/ipykernel_5140/348396083.py:27: RuntimeWarning: invalid value encountered in divide
  return ((1.0 - np.exp(-2.0 * x, dtype=np.float32)) /
/tmp/ipykernel_5140/348396083.py:33: RuntimeWarning: overflow encountered in exp
  return ((np.exp(x, dtype=np.float32) - np.exp(-x, dtype=np.float32))

In [4]:
def rank_techniques(x_fp64, session, input_name):
    # Reference
    y_ref = np.tanh(x_fp64)

    # input float32
    x_f32 = x_fp64.astype(np.float32)

    results = {
        "ONNX Tanh (ORT float32)": session.run(None, {input_name: x_f32})[0].astype(np.float32),
        "Technique 1 (branching: exp(2x) if x<0 else exp(-2x))": tanh_np_technique_1(x_f32),
        "Technique 2 (single formula exp(2x) / (exp(2x)+1))": tanh_np_technique_2(x_f32),
        "Technique 3 (single formula exp(-2x) / (1+exp(-2x)))": tanh_np_technique_3(x_f32),
        "Technique 4 (classic: (exp(x)-exp(-x))/(exp(x)+exp(-x)))": tanh_np_technique_4(x_f32),
        "Technique 5 (libm-like piecewise polynomial + exp clamp)": tanh_np_technique_5(x_f32),
        "Technique 6 (ONNX-like:Eigen/MLAS polynomial rational approximation)": tanh_np_technique_6(x_f32),
    }


    ranking = []

    for name, y_f32 in results.items():
        y_f64 = y_f32.astype(np.float64)

        abs_err = np.abs(y_f64 - y_ref)

        # ignore NaNs for metrics
        abs_err_clean = abs_err[np.isfinite(abs_err)]

        mean_abs = np.mean(abs_err_clean)
        max_abs = np.max(abs_err_clean)
        rms_abs = np.sqrt(np.mean(abs_err_clean**2))

        ranking.append((name, mean_abs, max_abs, rms_abs))

    # Sort by max error first, then mean, then rms
    ranking_sorted = sorted(ranking, key=lambda t: (t[2], t[1], t[3]))

    # Display
    print("\n==================== RANKING ====================")
    print("Best -> Worst (based on max abs error)")
    print("------------------------------------------------")
    print("Rank | Technique | mean_abs | max_abs | rms")
    for i, (name, mean_abs, max_abs, rms_abs) in enumerate(ranking_sorted, start=1):
        print(f"{i:>4d} | {name:<20s} | {mean_abs:.3e} | {max_abs:.3e} | {rms_abs:.3e}")

    return ranking_sorted


# ------------------------------------------------------------
# Stress test values
# ------------------------------------------------------------
test_values_fp64 = np.array([
    0.0,
    1e-8,
    -1e-8,
    1.0,
    -1.0,
    5.0,
    -5.0,
    10.0,
    -10.0,
    20.0,
    -20.0,
    50.0,
    -50.0,
    80.0,
    -80.0,
    88.0,
    -88.0,
    100.0,
    -100.0,
    1e3,
    -1e3,
    np.inf,
    -np.inf,
    np.nan
], dtype=np.float64)

x_test_fp64 = test_values_fp64.reshape(-1, 1)

# ------------------------------------------------------------
# Call ranking (replace session/input_name by ton ONNX session)
# ------------------------------------------------------------
rank_techniques(x_test_fp64, session, input_name)



==================== RANKING ====================
Best -> Worst (based on max abs error)
------------------------------------------------
Rank | Technique | mean_abs | max_abs | rms
   1 | Technique 5 (libm-like piecewise polynomial + exp clamp) | 5.326e-09 | 3.927e-08 | 1.278e-08
   2 | Technique 6 (ONNX-like:Eigen/MLAS polynomial rational approximation) | 5.757e-09 | 4.174e-08 | 1.375e-08
   3 | ONNX Tanh (ORT float32) | 5.757e-09 | 4.174e-08 | 1.375e-08
   4 | Technique 1 (branching: exp(2x) if x<0 else exp(-2x)) | 8.272e-09 | 4.174e-08 | 1.720e-08
   5 | Technique 2 (single formula exp(2x) / (exp(2x)+1)) | 8.674e-09 | 4.174e-08 | 1.581e-08
   6 | Technique 3 (single formula exp(-2x) / (1+exp(-2x))) | 8.674e-09 | 4.174e-08 | 1.581e-08
   7 | Technique 4 (classic: (exp(x)-exp(-x))/(exp(x)+exp(-x))) | 1.598e-08 | 1.013e-07 | 3.565e-08


/tmp/ipykernel_5140/348396083.py:21: RuntimeWarning: overflow encountered in exp
  return ((np.exp(2.0 * x, dtype=np.float32) - 1.0) /
/tmp/ipykernel_5140/348396083.py:22: RuntimeWarning: overflow encountered in exp
  (np.exp(2.0 * x, dtype=np.float32) + 1.0)).astype(np.float32)
/tmp/ipykernel_5140/348396083.py:21: RuntimeWarning: invalid value encountered in divide
  return ((np.exp(2.0 * x, dtype=np.float32) - 1.0) /
/tmp/ipykernel_5140/348396083.py:27: RuntimeWarning: overflow encountered in exp
  return ((1.0 - np.exp(-2.0 * x, dtype=np.float32)) /
/tmp/ipykernel_5140/348396083.py:28: RuntimeWarning: overflow encountered in exp
  (1.0 + np.exp(-2.0 * x, dtype=np.float32))).astype(np.float32)
/tmp/ipykernel_5140/348396083.py:27: RuntimeWarning: invalid value encountered in divide
  return ((1.0 - np.exp(-2.0 * x, dtype=np.float32)) /
/tmp/ipykernel_5140/348396083.py:33: RuntimeWarning: overflow encountered in exp
  return ((np.exp(x, dtype=np.float32) - np.exp(-x, dtype=np.float32))

[('Technique 5 (libm-like piecewise polynomial + exp clamp)',
  np.float64(5.326409521631861e-09),
  np.float64(3.926799019282612e-08),
  np.float64(1.2779296974320608e-08)),
 ('Technique 6 (ONNX-like:Eigen/MLAS polynomial rational approximation)',
  np.float64(5.756538671320927e-09),
  np.float64(4.174123280353825e-08),
  np.float64(1.3745840328355247e-08)),
 ('ONNX Tanh (ORT float32)',
  np.float64(5.756538748553834e-09),
  np.float64(4.174123280353825e-08),
  np.float64(1.374584032835525e-08)),
 ('Technique 1 (branching: exp(2x) if x<0 else exp(-2x))',
  np.float64(8.272306979971962e-09),
  np.float64(4.174123280353825e-08),
  np.float64(1.719782338856382e-08)),
 ('Technique 2 (single formula exp(2x) / (exp(2x)+1))',
  np.float64(8.67375906455339e-09),
  np.float64(4.174123280353825e-08),
  np.float64(1.5813198900882747e-08)),
 ('Technique 3 (single formula exp(-2x) / (1+exp(-2x)))',
  np.float64(8.67375906455339e-09),
  np.float64(4.174123280353825e-08),
  np.float64(1.581319890088